In [10]:
import numpy as np
from imblearn.over_sampling import SVMSMOTE
from sklearn.datasets import make_classification
from cuml.neighbors import NearestNeighbors
from cuml.svm import SVC

In [11]:
X_train = np.load("word2vec_train_vector.npy")
y_train = np.load("y_train.npy")
print(X_train.shape, y_train.shape)

(994372, 300) (994372,)


In [15]:
X_train[0]

array([-1.29525987e-02,  1.85034811e-01,  5.55096865e-02,  4.04699482e-02,
       -1.17009267e-01, -1.19585447e-01,  1.56577766e-01, -5.66713735e-02,
       -2.51536164e-03,  5.40111475e-02,  2.43222341e-02, -9.16895047e-02,
       -3.56970966e-01,  2.50710994e-01,  1.38686806e-01,  6.42760321e-02,
       -1.10400029e-01,  2.26538302e-03,  6.44911304e-02,  9.61638466e-02,
       -3.47243845e-02,  7.63154626e-02, -1.92960277e-02, -6.56623691e-02,
        2.96386145e-02, -1.00696720e-01,  6.94591850e-02, -1.33384347e-01,
       -1.56517550e-01,  9.63915139e-02, -2.10311145e-01, -1.02443546e-01,
       -1.63427159e-01, -6.86370805e-02, -5.51304743e-02,  1.41731292e-01,
        3.17338437e-01,  1.26478985e-01,  3.85488831e-02,  1.05231881e-01,
       -7.84558877e-02,  2.67572224e-01,  2.34290987e-01, -8.56749043e-02,
       -8.66069570e-02, -3.96924801e-02, -3.17438468e-02,  3.96249741e-02,
        3.88611704e-02, -1.84810266e-01,  7.47486800e-02,  2.50063926e-01,
       -2.83368498e-01, -

In [16]:
y_train[0]

1

In [13]:
X, y = make_classification(
  n_samples=100000,
  n_features=100,
  n_redundant=0,
  n_informative=100,
  n_classes=5,
  n_clusters_per_class=1,
  weights=[0.8, 0.05, 0.05, 0.05, 0.05]
)
print(X.shape, y.shape)

(100000, 100) (100000,)


In [14]:
X[0]

array([ -7.17844597,   5.06041077,   8.12363385,   7.75352782,
        -8.84328524,   3.92556313,  -3.17942985,   5.64604787,
        -0.74018225,  -2.18423471,  -3.54985533,  -4.30241996,
         7.5102416 ,   2.1283836 ,  11.23716761,  -9.26762869,
        -7.65870798,   3.5410399 ,  -8.00568321,  -1.04995983,
         3.65940375,   4.02911371,   0.14169811,   5.28412158,
         3.51510693, -11.27200465,  -3.57170497,  -0.64119809,
         8.67719188,   1.61573646,   2.19564017,  -1.55892497,
        -0.64664106,   3.85381569,  10.86210018,   4.12696723,
        12.59700974,   5.45640465,  -0.3841536 ,  -6.50424556,
         6.50273422,  -4.04159505,   8.04959346,   1.28220096,
        -6.46534164,  -2.40101731,  -1.07407933,   3.5993476 ,
        -1.54511647,  -0.9447397 ,  -1.53346904, -11.16081775,
         4.40225451,   3.08992658,  -2.11021693,   2.14480466,
        -3.39255598,  -3.92070297,   2.23690229,   2.95718583,
         0.17834081,   4.64900633,   3.97401054, -10.83

In [22]:
y[5]

0

In [26]:
np.random.seed(42)

# Specify the number of samples you want
n_samples = 10000  # Adjust this based on your GPU memory

# Generate random indices
indices = np.random.choice(X_train.shape[0], size=n_samples, replace=False)

# Create the sampled datasets
X_train_sampled = X_train[indices]
y_train_sampled = y_train[indices]

nn = NearestNeighbors(n_neighbors=6)
svm = SVC()
X_resampled, y_resampled = SVMSMOTE(
  k_neighbors=nn,
  m_neighbors=nn,
  svm_estimator=svm
).fit_resample(X_train_sampled, y_train_sampled)

[2025-04-07 17:37:00.916] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
[2025-04-07 17:37:00.916] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization


In [31]:
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.datasets import make_classification
from cuml.neighbors import NearestNeighbors

X, y = make_classification(
  n_samples=50000,
  n_features=200,
  n_redundant=0,
  n_informative=100,
  n_classes=2,
  n_clusters_per_class=1,
  weights=[0.9, 0.1]
)

nn = NearestNeighbors(n_neighbors=4)
enn = EditedNearestNeighbours(n_neighbors=nn)
X_res, y_res = enn.fit_resample(X, y)
print(X_res.shape, y_res.shape)

[2025-04-07 17:47:05.334] [CUML] [info] Unused keyword parameter: n_jobs during cuML estimator initialization
(49146, 200) (49146,)


In [ ]:
import numpy as np
import cupy as cp
from cuml.neighbors import KNeighborsClassifier
from collections import Counter
import random


class CUDACondensedNearestNeighbour:
    """
    CUDA-accelerated implementation of Condensed Nearest Neighbor algorithm
    for undersampling imbalanced datasets.
    """

    def __init__(self, n_neighbors=1, random_state=None):
        self.n_neighbors = n_neighbors
        self.random_state = random_state

    def fit_resample(self, X, y):
        """
        Resample X and y according to Condensed Nearest Neighbor algorithm.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data.
        y : array-like of shape (n_samples,)
            Target class labels.

        Returns
        -------
        X_resampled : array-like of shape (n_samples_new, n_features)
            Resampled training data.
        y_resampled : array-like of shape (n_samples_new,)
            Resampled target class labels.
        """
        # Set random seed for reproducibility
        if self.random_state is not None:
            random.seed(self.random_state)
            np.random.seed(self.random_state)

        # Initialize the KNN classifier
        if isinstance(self.n_neighbors, int):
            knn = KNeighborsClassifier(n_neighbors=self.n_neighbors)
        else:
            knn = self.n_neighbors  # Use the provided classifier

        # Get class information
        classes = np.unique(y)
        class_indices = {c: np.where(y == c)[0] for c in classes}

        # Step 1: Initialize store set S with one random sample from each class
        store_indices = []
        for c in classes:
            store_indices.append(random.choice(class_indices[c]))

        # Convert to numpy arrays
        X_store = X[store_indices]
        y_store = y[store_indices]

        # Step 2: Iteratively add misclassified samples
        added = True
        remaining_indices = np.setdiff1d(np.arange(len(X)), store_indices)

        while added:
            added = False

            # Fit the KNN classifier on the current store set
            knn.fit(X_store, y_store)

            # Find samples that would be misclassified
            X_remaining = X[remaining_indices]
            y_remaining = y[remaining_indices]

            # Predict in batches to avoid memory issues
            batch_size = 10000  # Adjust based on your GPU memory
            n_batches = int(np.ceil(len(X_remaining) / batch_size))
            y_pred = np.zeros(len(X_remaining), dtype=y.dtype)

            for i in range(n_batches):
                start_idx = i * batch_size
                end_idx = min((i + 1) * batch_size, len(X_remaining))
                y_pred[start_idx:end_idx] = knn.predict(X_remaining[start_idx:end_idx])

            # Identify misclassified samples
            misclassified = y_pred != y_remaining

            if np.any(misclassified):
                # Add first misclassified sample to the store set
                misclassified_idx = np.where(misclassified)[0][0]
                original_idx = remaining_indices[misclassified_idx]

                # Update store set
                X_store = np.vstack([X_store, X[original_idx:original_idx+1]])
                y_store = np.append(y_store, y[original_idx])

                # Update remaining indices
                remaining_indices = np.delete(remaining_indices, misclassified_idx)

                added = True

        return X_store, y_store

import numpy as np
from cuml.neighbors import KNeighborsClassifier
# Import the custom class from wherever you saved it
# from cuda_cnn import CUDACondensedNearestNeighbour

# Load your data
X_train = np.load("word2vec_train_vector.npy")
y_train = np.load("y_train.npy")

# Create a cuML KNN classifier
knn = KNeighborsClassifier(n_neighbors=1)

# Option 1: Pass the KNN instance directly
cnn = CUDACondensedNearestNeighbour(n_neighbors=knn)

# Option 2: Or just specify the number of neighbors
# cnn = CUDACondensedNearestNeighbour(n_neighbors=1)

# Perform undersampling
X_res, y_res = cnn.fit_resample(X_train, y_train)

print(f"Original dataset shape: {X_train.shape}")
print(f"Resampled dataset shape: {X_res.shape}")